In [2]:
import numpy as np
import pandas as pd

In [48]:
events = pd.read_csv('data/events_elo.csv')
events.head()

,game_dt,bat_id,pit_id,event_cd,bunt_fl,batter_score,pitcher_score
0,20130409,crisc001,wilsc004,2,F,0.0,1.0
1,20130409,younc004,wilsc004,2,F,0.0,1.0
2,20130409,lowrj001,wilsc004,14,F,1.0,0.0
3,20130409,cespy001,wilsc004,14,F,1.0,0.0
4,20130409,norrd001,wilsc004,20,F,1.0,0.0


In [49]:
elo_table = pd.read_csv('data/elo.csv')
elo_table.head()

,player_id,last_name_tx,first_name_tx,team_id,team_tx,pos_tx,elo_rating
0,zimmr001,Zimmerman,Ryan,WAS,WAS,3B,1500
1,zimmj003,Zimmermann,Jordan,WAS,WAS,P,1500
2,wertj001,Werth,Jayson,WAS,WAS,OF,1500
3,waltz001,Walters,Zach,WAS,WAS,SS,1500
4,tracc001,Tracy,Chad,WAS,WAS,3B,1500


In [50]:
def get_current_elo(player, elo_table):
    return elo_table.loc[elo['player_id'] == player]['elo_rating'].values[0]

def calc_expected(player1, player2, elo_table):
    p1_diff = get_current_elo(player1, elo_table) - get_current_elo(player2, elo_table)
    p2_diff = -p1_diff
    
    return 1/(1+(10**(p1_diff/400))), 1/(1+(10**(p2_diff/400)))

In [51]:
calc_expected('zimmr001', 'zimmj003', elo)

(0.5, 0.5)

In [117]:
def update_elo(player1, player2, p1_score, p2_score, elo_table, player_index, k=32):
    exp_p1, exp_p2 = calc_expected(player1, player2, elo_table)
    p1_curr = get_current_elo(player1, elo_table)
    p2_curr = get_current_elo(player2, elo_table)
    
    p1_new = p1_curr+(k*(p1_score-p1_curr))
    p2_new = p2_curr+(k*(p2_score-p2_curr))
    
    
    elo_table.iat[player_index[player1], 'elo_rating'] = p1_new
    elo_table.iat[player_index[player2], 'elo_rating'] = p2_new

In [115]:
def season_elo(events, elo_table, k=32):
    player_index = dict(zip(elo_table['player_id'], range(elo_table.shape[0])))
    
    for _, row in events.iterrows():
        update_elo(row['bat_id'], row['pit_id'], row['batter_score'], row['pitcher_score'], elo_table, player_index, k)

In [116]:
season_elo(events, elo_table)

/home/odin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: overflow encountered in double_scalars
  


KeyboardInterrupt: 